<a href="https://colab.research.google.com/github/alexlinapp/python_tools_practice/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install fsspec==2023.6.0
!pip install --upgrade transformers tokenizers
# !pip uninstall -y torch torchtext
# !pip install torch==2.0.1+cu117 torchtext==0.15.2 -f https://download.pytorch.org/whl/torch_stable.html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.6.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64

In [ ]:
# from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from collections import Counter
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from transformers import GPT2Tokenizer
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.utils.data import TensorDataset
print(torch.__version__)

2.6.0+cu124


In [ ]:
from datasets import load_dataset

# This should load without issues
dataset = load_dataset("wikitext", "wikitext-2-v1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
inputs = tokenized['train']['input_ids']
print(type(tokenized['train']))

from itertools import chain

# Suppose 'tokenized' is your tokenized DatasetDict or Dataset

# Extract the list of token lists (input_ids) from your train split
input_ids_lists = tokenized['train']['input_ids']  # list of lists of ints

# Use itertools.chain to flatten efficiently into one big list
all_input_ids = list(chain.from_iterable(input_ids_lists))


<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
def group_strides_tensor(seq, bs):
  n = seq.size(0) // bs
  x = seq[:bs * n].reshape(bs, n, -1)


In [ ]:
input_tokens = torch.tensor(all_input_ids)[:200000]
bs = 64
seq_len = 5;
x = input_tokens.unfold(0, seq_len, 1)[:-1] # drops the last window
y = input_tokens[seq_len:]

tokendataset = TensorDataset(x, y)
dataloader = DataLoader(tokendataset, batch_size=bs, shuffle=True, drop_last=True)
first = next(iter(dataloader))
first[0].shape, first[1].shape
m1 = LMModel1(vocab_sz=tokenizer.vocab_size, n_hidden=16, seq_len = seq_len, n_layers=1)
#m1(first[0]).shape
# grouping by strides
print(first[0].shape)
m2 = LSTMCell(16, 16, tokenizer.vocab_size, 5, "cuda")
m2(first[0]).shape


torch.Size([64, 5])


torch.Size([64, 50257])

In [ ]:
class LMModel1(nn.Module):
  def __init__(self, vocab_sz, n_hidden, seq_len, n_layers=1):
    super().__init__()
    self.i_h = nn.Embedding(vocab_sz, n_hidden) # Embedded layer. Maps inputs (vocab) to vector
    self.h_h = nn.Linear(n_hidden, n_hidden)    # Hidden layer
    self.rnn = nn.RNN(n_hidden, n_hidden, n_layers, batch_first=True)
    self.h_o = nn.Linear(n_hidden, vocab_sz)    # Output layer
    self.seq_len = seq_len
    self.h = torch.zeros(n_layers, bs, n_hidden).to("cuda")
  def forward(self, x):
    # for i in range(self.seq_len):
    #   self.h = self.h + self.i_h(x[:,i])
    #   #print(self.h.shape)
    #   self.h = F.relu(self.h_h(self.h))
    #   out = self.h_o(self.h)
    #   print(out.shape)
    #   self.h = self.h.detach()
    # return out
    res,h = self.rnn(self.i_h(x), self.h)
    #print(res.shape)
    self.h = h.detach()
    #print(res.shape)
    return self.h_o(res[:,-1,:])
  def reset(self): self.h = 0

class LSTMCell(nn.Module):
    def __init__(self, ni, nh, vocab_size, seq_len, device):
        super().__init__()
        self.ih = nn.Linear(ni, 4 * nh)
        self.hh = nn.Linear(nh, 4 * nh)
        self.i_he = nn.Embedding(vocab_size, ni)
        self.h_o = nn.Linear(nh, vocab_size)
        self.seq_len = seq_len
        self.h = torch.zeros(bs, nh).to(device)
        self.c = torch.zeros(bs, nh).to(device)
    def forward(self, input):
        for i in range(self.seq_len):
          input_step = self.i_he(input[:,i])
          gates = (self.ih(input_step) + self.hh(self.h)).chunk(4, 1)
          ingate, forgetgate, outgate = map(torch.sigmoid, gates[:3])
          cellgate = gates[3].tanh()
          self.c = (forgetgate * self.c) + (ingate * cellgate)
          self.h = outgate * self.c.tanh()
        self.h = self.h.detach()
        self.c = self.c.detach()
        return self.h_o(self.h)

In [ ]:
def train_model(model, dataloader, optimizer, loss_fnc, device, epochs=4):
  model.to(device)
  model.train()
  for epoch in range(epochs):
    total_loss = 0
    total_correct = 0
    total_tokens = 0
    print("entering")
    for xb, yb in dataloader:
      xb = xb.to(device)
      yb = yb.to(device)
      optimizer.zero_grad()
      out = model(xb)        # shape: (batch_size, sequence_length)
      #print(out.shape)
      loss = loss_fnc(out, yb)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
      total_correct += (out.argmax(dim=1) == yb).sum().item()
      total_tokens += xb.shape[0]
    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_tokens
    print(f"Epoch {epoch+1}/{epochs} — Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

In [ ]:
model = LMModel1(vocab_sz=tokenizer.vocab_size, n_hidden=32, seq_len = seq_len, n_layers=4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fnc = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_model(model, dataloader, optimizer, loss_fnc, device, epochs=4)

entering
Epoch 1/4 — Loss: 6.7006, Accuracy: 0.1374
entering
Epoch 2/4 — Loss: 5.9176, Accuracy: 0.1877
entering
Epoch 3/4 — Loss: 5.6553, Accuracy: 0.2004
entering
Epoch 4/4 — Loss: 5.4581, Accuracy: 0.2112


In [ ]:
model2 = LSTMCell(16, 16, tokenizer.vocab_size, seq_len, device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fnc = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_model(model2, dataloader, optimizer, loss_fnc, device, epochs=4)

entering
Epoch 1/4 — Loss: 10.8542, Accuracy: 0.0000
entering
Epoch 2/4 — Loss: 10.8542, Accuracy: 0.0000
entering


KeyboardInterrupt: 

In [ ]:
import time
start = time.time()
for xb, yb in dataloader:
    pass
print(f"DataLoader iteration took {time.time()-start:.2f} seconds")

DataLoader iteration took 19.69 seconds
